In [ ]:
import sys
if '..' not in sys.path: sys.path.append('..')

In [ ]:
from datasets import load_dataset

ds = load_dataset('openai/gsm8k', 'main')

In [ ]:
from utils import BT
from tqdm import tqdm

q_len = []
a_len = []

for thing in tqdm(ds['train']['question']):
    q_len.append(len(BT(thing)['input_ids']))
    
for thing in tqdm(ds['train']['answer']):
    a_len.append(len(BT(thing)['input_ids']))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(q_len)
plt.hist(a_len)
plt.hist([q+a for q, a in zip(q_len, a_len)])

# context length 512 will fit

In [ ]:
import re
import json
from tqdm import tqdm

def aux(q, a):
    a = re.sub(r'<<.*?>>', '', a)
    a_toks = BT(a, max_length=512, truncation=True)['input_ids'][1:-1]
    a_traj = []
    
    while a_toks:
        a_traj.append(BT.decode(a_toks[:]))
        a_toks = a_toks[:len(a_toks)//2]
    
    return [q] + [f'{q} {a}' for a in a_traj][::-1]

train_val_split = ds['train'].train_test_split(test_size=0.1)
train = train_val_split['train']
val = train_val_split['test']

with open('../data/gsm/gsm_train_1.jsonl', 'w') as f:
    for x in tqdm(train):
        q = x['question']
        a = x['answer']
        json.dump([aux(x['question'], x['answer']), 0], f)
        f.write('\n')
        
with open('../data/gsm/gsm_dev_1.jsonl', 'w') as f:
    for x in tqdm(val):
        q = x['question']
        a = x['answer']
        json.dump([aux(x['question'], x['answer']), 0], f)
        f.write('\n')

In [ ]:
from utils import generate_prefix_alignment

generate_prefix_alignment(
    input_path='../data/gsm/gsm_train_1.jsonl',
    output_dir='../data/gsm/cache_1',
    cache_prefix='sup-gsm-1',
    max_len=512
)